In [16]:
import cv2
import numpy as np

WIND_X = 1300
WIND_Y = 700

class Figure:
    def __init__ (self, x_, y_, vx_, vy_):
        self.x = x_
        self.y = y_
        self.vx = vx_
        self.vy = vy_
    
    def move (self):
        self.x, self.vx = self._move (self.x, self.vx, 0, WIND_X)
        self.y, self.vy = self._move (self.y, self.vy, 0, WIND_Y)
    
    def _move (self, x, vx, xmi, xma):
        x += vx
        
        if (x <= xmi or x >= xma):
            vx *= -1
        
        return x, vx
    
    def draw (self, canvas):
        #рисование объекта
        pass

class Circle(Figure):
    def __init__ (self, x_, y_, vx_, vy_, r_):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.r = r_
    
    def draw (self, canvas):
        cv2.circle (canvas, (self.x, self.y), self.r, (100, (50 + self.y * 3) % 255, (50 + self.x * 2) % 255), -1)
        
    def move (self):
        self.x, self.vx = self._move (self.x, self.vx, self.r, WIND_X - self.r)
        self.y, self.vy = self._move (self.y, self.vy, self.r, WIND_Y - self.r)

class Square(Figure):
    def __init__ (self, x_, y_, vx_, vy_, sz_):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.sz = sz_
    
    def draw (self, canvas):
        cv2.rectangle (canvas, (self.x, self.y), (self.x + self.sz, self.y + self.sz), (200, 20, 50), -1)
        
    def move (self):
        self.x, self.vx = self.move (self.x, self.vx, 0, WIND_X - self.sz)
        self.y, self.vy = self.move (self.y, self.vy, 0, WIND_Y - self.sz)
        
class Line(Figure):
    def __init__ (self, x1_, y1_, x2_, y2_, vx1_, vy1_, vx2_, vy2_):
        Figure.__init__ (self, x1_, y1_, vx1_, vy1_)
        
        self.fp = Figure (x1_, y1_, vx1_, vy1_) #first point
        self.sp = Figure (x2_, y2_, vx2_, vy2_) #second point
    
    def draw (self, canvas):
        cv2.line (canvas, (self.fp.x, self.fp.y), (self.sp.x, self.sp.y),
            ((200 + int (self.fp.y * 0.2)) % 255,
             (200 + int (self.fp.x * 0.4)) % 255,
             (int (20 + self.sp.y * 0.7)) % 255), 2)
        
    def move (self):
        self.fp.move ()
        self.sp.move ()

#холст
canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 60

figures = []

#figures.append (Circle (100, 200, -1, 3, 30))
#figures.append (Square (100, 200, 2, -3, 40))
figures.append (Line (100, 200, 200, 300, 2, -4, 4, -3))

while (True):
    #очистка холста
    #canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 200
    
    #движение
    for fig in figures:
        fig.move ()
    
    #рисование
    for fig in figures:
        fig.draw (canvas)    
    
    cv2.imshow ("frame", canvas)
    
    #выход из цикла
    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

    if (keyb == ord('a')):
        vx = np.random.randint (1, 5)
        vy = -np.random.randint (1, 5)
        r = np.random.randint (20, 50)
        
        figures.append (Circle (100, 200, vx, vy, r))
    
cv2.destroyAllWindows ()